In [22]:
import pandas as pd
import os
import os.path
import matplotlib.pyplot as plt
import sklearn as skl
import numpy as np
import scipy 
#import PyQt5
from matplotlib.colors import BoundaryNorm
import matplotlib.ticker as ticker
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.ticker import MaxNLocator
from scipy.signal import find_peaks
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from AllPackage.PreProcessing import *

In [23]:
# leggo i file contunuti nella cartella Campioni_S_11x11 e acquisisco i nomi dei file
pathCampioni = '../Campioni _S_11x11/'
namefiles = [f for f in os.listdir(pathCampioni) if os.path.isfile(os.path.join(pathCampioni, f))]
namefiles

['S1_bkg_mapA_11x11.txt',
 'S1_mapA_11x11.txt',
 'S2_bkg_mapA_11x11.txt',
 'S2_mapA_11x11.txt']

In [3]:
# creo la variabile contenente i nomi delle colonne del pandas dataframe

columns = ['WaveNum']+[f'row{k}_point{i}' for k in range(1,12) for i in range(1,12)]


In [4]:
# creo i pandas dataframe dai file contenuti nella cartella Campioni_S_11x11

#for name in namefiles:
#    globals()['dataset_%s' % name[:-4]] = pd.read_csv('./Campioni _S_11x11/'+namefiles[0], delim_whitespace=True,names = columns)
    
data_S1_bkg = pd.read_csv(pathCampioni+namefiles[0], delim_whitespace=True,names = columns)
data_S1 = pd.read_csv(pathCampioni+namefiles[1], delim_whitespace=True,names = columns)
data_S2_bkg = pd.read_csv(pathCampioni+namefiles[2], delim_whitespace=True,names = columns)
data_S2 = pd.read_csv(pathCampioni+namefiles[3], delim_whitespace=True,names = columns)



In [5]:
#proviamo a fare un po' di preprocessing

data_S1_bkg_smooth = smooth_filter_savgol(data_S1_bkg,data_S1_bkg.columns)

#plot_spettri_pdf(S1_bkg_smooth,S1_bkg_smooth.columns,'./smooth_pdf/')
#plot_spettri_pdf(data_S1_bkg,data_S1_bkg.columns,'./pdf/')


#[peaks_wn, peaks_int] = trova_picchi(data_S1_bkg_smooth,data_S1_bkg.columns)

data_S1_bkg_smooth_Mm = norm_max_min(data_S1_bkg,data_S1_bkg.columns)
#il trovapicchi sembra essere indip. da l'uso dello smooth


#prova di trasposizione
S1_bkg_trasp = transpose_df(data_S1_bkg_smooth_Mm)

In [6]:
"""
index = (8-1)*11 + 1 -1 # (row-1)*11 + point -1

fig = plt.figure();
    
ax = fig.add_subplot(1, 1, 1);

ax.plot(data_S1_bkg_smooth[columns[0]],  data_S1_bkg_smooth.row8_point1);

ax.scatter(peaks_wn[index],peaks_int[index],color='r',marker='x')

ax.grid(which='minor', linestyle=':', linewidth=0.5);

ax.minorticks_on();

#ax.set_title(S1_bkg_smooth.columns[24]);
ax.set_ylabel('Raman intensity')
ax.set_xlabel('Raman shift (cm$^{-1}$)')
"""

"\nindex = (8-1)*11 + 1 -1 # (row-1)*11 + point -1\n\nfig = plt.figure();\n    \nax = fig.add_subplot(1, 1, 1);\n\nax.plot(data_S1_bkg_smooth[columns[0]],  data_S1_bkg_smooth.row8_point1);\n\nax.scatter(peaks_wn[index],peaks_int[index],color='r',marker='x')\n\nax.grid(which='minor', linestyle=':', linewidth=0.5);\n\nax.minorticks_on();\n\n#ax.set_title(S1_bkg_smooth.columns[24]);\nax.set_ylabel('Raman intensity')\nax.set_xlabel('Raman shift (cm$^{-1}$)')\n"

In [7]:
pca = skl.decomposition.PCA(n_components=3)
StandardScaler = skl.preprocessing.StandardScaler()

In [8]:
StandardScaler.fit_transform(S1_bkg_trasp)
asd = pca.fit(S1_bkg_trasp)

In [9]:
wow = asd.transform(S1_bkg_trasp)
asd.explained_variance_ratio_

array([0.63518543, 0.09919981, 0.04668257])

In [10]:
#esp (la distanza tra punti interni) è il parametro più influente con cui giocare

In [11]:
# cluster è un array contenente l'indice del cluster di appartenenza di ogni punto del dataframe
cluster = DBSCAN(eps=1.5, min_samples=5).fit_predict(wow)
cluster

array([ 0,  1, -1,  2,  2,  2,  2,  2,  0,  1,  2,  2,  2,  2,  2, -1,  2,
        0,  2,  2,  2,  2,  2,  2,  2,  2, -1,  3, -1,  2,  2, -1, -1,  2,
       -1,  1,  2,  2, -1,  2,  2,  2,  2,  2,  3,  0,  2, -1, -1,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2, -1,  3,  1,  1,  0, -1,
        3,  1,  2,  2, -1,  2, -1,  2,  0,  2,  3,  2,  2, -1,  2,  2,  2,
        1, -1, -1,  1,  2,  2,  2, -1,  1,  2, -1,  2,  1, -1,  2,  2,  2,
        2,  2, -1,  2,  2,  2,  2, -1,  2, -1,  2,  2,  1,  2, -1, -1, -1,
       -1,  1], dtype=int64)

In [12]:
color = []
for x in cluster:
    if(x==0):
        color.append('tab:blue')
    if(x==1):
        color.append('tab:orange')
    if(x==2):
        color.append('tab:green')
    if(x==3):
        color.append('tab:red' )
    if(x==-1):
        color.append('tab:purple')


In [13]:
%matplotlib qt

fig = plt.figure();

X=wow[:,0]
Y=wow[:,1]
Z=wow[:,2]

ax = fig.add_subplot(111,projection='3d');
ax.scatter(X, Y, Z, c=color, marker='x')
ax.grid(which='minor', linestyle=':', linewidth=0.5);

ax.minorticks_on();

ax.set_title(data_S1_bkg.columns[24]);
ax.set_ylabel('PC1')
ax.set_xlabel('PC0')
ax.set_zlabel('PC2')

plt.show()